In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import sqlite3

In [2]:
# read csv & clean data
lifecsv = pd.read_csv("Life Expectancy (years) by Country.csv", encoding='latin1')

lifecsv = lifecsv.drop(columns=['Series Name','Series Code'])

lifecsv = lifecsv.rename(columns={'1999 [YR1999]':'1999','2000 [YR2000]':'2000','2001 [YR2001]':'2001','2002 [YR2002]':'2002','2003 [YR2003]':'2003','2004 [YR2004]':'2004','2005 [YR2005]':'2005','2006 [YR2006]':'2006','2007 [YR2007]':'2007','2008 [YR2008]':'2008','2009 [YR2009]':'2009','2010 [YR2010]':'2010','2011 [YR2011]':'2011','2012 [YR2012]':'2012','2013 [YR2013]':'2013','2014 [YR2014]':'2014','2015 [YR2015]':'2015','2016 [YR2016]':'2016','2017 [YR2017]':'2017'})

lifecsv = lifecsv.dropna()

In [3]:
lifecsv.head()

,Country Name,Country Code,1999,2000,2001,2002,2003,2004,2005,2006,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,AFG,54.959,55.482,56.044,56.637,57.25,57.875,58.5,59.11,...,60.243,60.754,61.226,61.666,62.086,62.494,62.895,63.288,63.673,64.047
1,Albania,ALB,73.588,73.955,74.286,74.575,74.82,75.028,75.217,75.418,...,75.943,76.281,76.652,77.031,77.389,77.702,77.963,78.174,78.345,78.495
2,Algeria,DZA,69.822,70.292,70.778,71.276,71.78,72.281,72.767,73.225,...,74.032,74.374,74.676,74.944,75.187,75.418,75.641,75.86,76.078,76.293
3,American Samoa,ASM,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,Andorra,AND,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [4]:
engine = create_engine("sqlite:///life.sqlite")
conn = engine.connect()

In [5]:
# lifecsv.to_sql(name='life', con=engine, if_exists='replace', index=True)

In [8]:
inspector = inspect(engine)
inspector.get_table_names()

['life']

In [9]:
data = pd.read_sql("SELECT * FROM life", conn)
data.head()

,id,Country_Name,Country_Code,Yr1999,Yr2000,Yr2001,Yr2002,Yr2003,Yr2004,Yr2005,...,Yr2008,Yr2009,Yr2010,Yr2011,Yr2012,Yr2013,Yr2014,Yr2015,Yr2016,Yr2017
0,0,Afghanistan,AFG,54.959,55.482,56.044,56.637,57.25,57.875,58.5,...,60.243,60.754,61.226,61.666,62.086,62.494,62.895,63.288,63.673,64.047
1,1,Albania,ALB,73.588,73.955,74.286,74.575,74.82,75.028,75.217,...,75.943,76.281,76.652,77.031,77.389,77.702,77.963,78.174,78.345,78.495
2,2,Algeria,DZA,69.822,70.292,70.778,71.276,71.78,72.281,72.767,...,74.032,74.374,74.676,74.944,75.187,75.418,75.641,75.86,76.078,76.293
3,3,American Samoa,ASM,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,4,Andorra,AND,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [10]:
engine = create_engine("sqlite:///life.sqlite")
session = Session(engine)

In [11]:
Base = automap_base()

In [12]:
Base.prepare(engine, reflect=True)

In [13]:
Base.classes.keys()

['life']

In [14]:
columns = inspector.get_columns('life')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
Country_Name TEXT
Country_Code TEXT
Yr1999 FLOAT
Yr2000 FLOAT
Yr2001 FLOAT
Yr2002 FLOAT
Yr2003 FLOAT
Yr2004 FLOAT
Yr2005 FLOAT
Yr2006 FLOAT
Yr2007 FLOAT
Yr2008 FLOAT
Yr2009 FLOAT
Yr2010 FLOAT
Yr2011 FLOAT
Yr2012 FLOAT
Yr2013 FLOAT
Yr2014 FLOAT
Yr2015 FLOAT
Yr2016 FLOAT
Yr2017 FLOAT


In [15]:
Life = Base.classes.life

In [16]:
first_row = session.query(Life).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x19adb4414e0>,
 'Yr1999': 54.959,
 'Yr2015': 63.288,
 'Yr2007': 59.694,
 'Yr2000': 55.482,
 'Yr2016': 63.673,
 'Yr2008': 60.243,
 'Yr2001': 56.044,
 'Yr2017': 64.047,
 'Yr2009': 60.754,
 'Yr2002': 56.637,
 'Yr2010': 61.226,
 'Yr2003': 57.25,
 'Yr2011': 61.666,
 'Yr2004': 57.875,
 'id': 0,
 'Yr2012': 62.086,
 'Yr2005': 58.5,
 'Country_Name': 'Afghanistan',
 'Yr2013': 62.494,
 'Yr2006': 59.11,
 'Country_Code': 'AFG',
 'Yr2014': 62.895}

In [18]:

for row in session.query(Life.Country_Name, Life.Yr1999, Life.Yr2017).limit(15).all():
    print(row)

('Afghanistan', 54.959, 64.047)
('Albania', 73.588, 78.495)
('Algeria', 69.822, 76.293)
('American Samoa', '..', '..')
('Andorra', '..', '..')
('Angola', 46.105, 61.809)
('Antigua and Barbuda', 73.327, 76.519)
('Argentina', 73.618, 76.738)
('Armenia', 70.944, 74.782)
('Aruba', 73.738, 76.01)
('Australia', 78.93170732, 82.49756098)
('Austria', 77.87560976, 81.64146341)
('Azerbaijan', 66.491, 72.123)
('Bahamas, The', 72.093, 75.823)
('Bahrain', 74.239, 77.038)
